In [62]:
import numpy as np
from tensorflow.keras.layers import Input, Dense, Dropout, Input, Conv2D, MaxPooling2D, Flatten, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical, plot_model

In [63]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [64]:
num_labels = len(np.unique(y_train))
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [65]:
image_size = x_train.shape[1]
x_train = np.reshape(x_train,[-1, image_size, image_size, 1])
x_test = np.reshape(x_test,[-1, image_size, image_size, 1])
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

In [66]:
input_shape = (image_size, image_size, 1)
batch_size = 32
kernel_size = 3
dropout = 0.4
n_filters = 32

In [67]:
# Left branch of Y network

left_inputs = Input(shape=input_shape)
x = left_inputs
filters = n_filters

for i in range(3):
    x = Conv2D(filters=filters,
               kernel_size=kernel_size,
               padding='same',
               activation='relu')(x)
    
    x = Dropout(dropout)(x)
    x = MaxPooling2D()(x)
    filters *= 2

In [68]:
# Right brancg of Y network

right_inputs = Input(shape=input_shape)
y = right_inputs
filters = n_filters

for i in range(3):
    y = Conv2D(filters=filters,
    kernel_size=kernel_size,
    padding='same',
    activation='relu',
    dilation_rate=2)(y)
    
    y = Dropout(dropout)(y)
    y = MaxPooling2D()(y)
    filters *= 2

In [69]:
y = concatenate([x, y])
y = Flatten()(y)
y = Dropout(dropout)(y)
outputs = Dense(num_labels, activation='softmax')(y)

In [70]:
model = Model([left_inputs, right_inputs], outputs)

In [71]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit([x_train, x_train], y_train, 
          validation_data=([x_test, x_test], y_test),
          epochs=20,
          batch_size=batch_size)

Epoch 1/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.1775 - accuracy: 0.9437 - val_loss: 0.1486 - val_accuracy: 0.9878
Epoch 2/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0619 - accuracy: 0.9807 - val_loss: 0.1093 - val_accuracy: 0.9889
Epoch 3/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0503 - accuracy: 0.9843 - val_loss: 0.0666 - val_accuracy: 0.9923
Epoch 4/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0466 - accuracy: 0.9857 - val_loss: 0.0622 - val_accuracy: 0.9935
Epoch 5/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0417 - accuracy: 0.9870 - val_loss: 0.0717 - val_accuracy: 0.9929
Epoch 6/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0378 - accuracy: 0.9884 - val_loss: 0.0609 - val_accuracy: 0.9931
Epoch 7/20
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0364 - accuracy: 0.9887 - val_loss: 0.0431 - val_ac

In [72]:
score = model.evaluate([x_test, x_test],
                       y_test,
                       batch_size=batch_size,
                       verbose=0)

print("\nTest accuracy: %.1f%%" % (100.0 * score[1]))


Test accuracy: 99.3%
